In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import sys
sys.path.append("../")
from src.Structure import AtomCls, AtmosphereCls
from src.Atomic import LTELib, ColExcite, SEsolver
#from src.RadiativeTransfer import Thin

---

# steps to use an `atom` class

In [ ]:
folder = "../data/atom/Ca_II/"
file     = folder + "Ca_II.Level"
file_Aji = folder + "Einstein_A/Nist.Aji"
file_CEe = folder + "Collisional_Excitation/RH.Electron"
file_CIe = folder + "Collisional_Ionization/RH.Electron"
file_PI  = folder + "Photoionization/RH.Alpha"
file_Rad = folder + "Ca_II.RadiativeLine"
atom = AtomCls.Atom(file, _file_Aji=file_Aji, _file_CEe=file_CEe)
atom.read_CI(_path_electron=file_CIe)
atom.read_PI(_path_alpha=file_PI)
atom.read_Mesh(_path=file_Rad)
atom.make_Mesh()
atom.read_Radiative_Line_intensity(_folder="../data/intensity/Ca_II/")

In [ ]:
del atom

minimum request : 
- level information
- Einstein Aji
- Collisional excitation

In [ ]:
folder = "../data/atom/Ca_II/"
file     = folder + "Ca_II.Level"
file_Aji = folder + "Einstein_A/Nist.Aji"
file_CEe = folder + "Collisional_Excitation/RH.Electron"
atom = AtomCls.Atom(file, _file_Aji=file_Aji, _file_CEe=file_CEe)

In [ ]:
atom.Z, atom.Element

Level information

In [ ]:
atom.Level

Line information

In [ ]:
atom.Line

Level and Line indentification (**important**)

In [ ]:
atom.Level_info_table

collisional excitation coeficient and table for interpolation

In [ ]:
atom.CE_coe, atom.CE_table

collisional ionization coeficient and table for interpolation

In [ ]:
file_CIe = folder + "Collisional_Ionization/RH.Electron"
atom.read_CI(_path_electron=file_CIe)

In [ ]:
atom.CI_coe, atom.CI_table

photoionization cross section and table for interpolation

In [ ]:
file_PI  = folder + "Photoionization/RH.Alpha"
atom.read_PI(_path_alpha=file_PI)

In [ ]:
atom.PI_coe

In [ ]:
atom.PI_table.shape

read the necessary parameter to construct line mesh

In [ ]:
atom.read_Mesh(_path=file_Rad)

In [ ]:
atom.Mesh_coe

construct line and continuum mesh

In [ ]:
atom.make_Mesh()

In [ ]:
print( "totally ", len(atom.line_mesh_list), " set of radiative mesh in the list" )

In [ ]:
mesh = atom.line_mesh_list[0]
plt.plot(mesh, "*")

In [ ]:
mesh = atom.continuum_mesh_list[0]
plt.plot(mesh, "*")

read incident background line intensity profile

In [ ]:
atom.read_Radiative_Line_intensity(_folder="../data/intensity/Ca_II/")

In [ ]:
arr = atom.radiative_line_intensity_list[0]

plt.plot(arr[0,:], arr[1,:], "*")

---

# steps to use `AtmosphereC1d` class

In [ ]:
slab1d = AtmosphereCls.AtmosphereC1d(1E7) # 1E7[cm] = 100[km]
slab1d.setUniform()
slab1d.readBackgroundIntensity('../data/intensity/continuum/ASTM_E490+PROM7_HI_CaII.dat')

`Mesh` is in relative scale [0,1]

In [ ]:
plt.plot( slab1d.data.Mesh, '*' )

In [ ]:
plt.plot( slab1d.data.Spatial, '*' )

- `Te` : electron temperature
- `Ne` : electron density
- `Vt` : turbulent velocity
- `Vd` : Doppler velocity
- `Pg` : gas pressure

In [ ]:
slab1d.data.Te # Ne, Vt, Vd, Pg

---

## not yet completed

solve Full Statistical Equilirbium at a single depth point

In [ ]:
def one_depth(_atom, _Te, _Ne, _Vt, _Vd):
    r"""
    calculate the 
        - LTE population
        - SE population (SE : Statistical Equilibrium)
        - SE population of the upper level of each transition
        - C and R matrice while solving SE
        - optically thin relative flux (some constants are removed)

    Parameters
    ----------

    _atom  : AtomCls.Atom() object
        atomic model data structure,
        number of Levels : _atom.nLevel
        number of Line transitions : _atom.nLine
        
    _Te  : scalar variable (np.float, np.double and so on)
        electron temperature, [:math:`K`]
        
    _Ne : scalar variable (np.float, np.double and so on)
        electron density, [:math:`cm^{-3}`]

    Returns
    -------

    _n_LTE : np.double, np.array with shape (_atom.nLevel,)
        Normalized LTE level population.
        :math:`rt = n_j / n_i`, [-]
    
    _n_SE : np.double, np.array with shape (_atom.nLevel,)
        Normalized level population under statistical equilibrium.
        
    _nj_SE : np.double, np.array with shape (_atom.nLine)
        Normalized level population under statistical equilibrium 
        of the upper levels of each line transition.
        
    _Cmat : np.double, np.array with shape (_atom.nLevel, _atom.nLevel)
        C matrix (Collisional related) in solving statistical equilibrium equations.
        
    _Rmat : np.double, np.array with shape (_atom.nLevel, _atom.nLevel)
        R matrix (Radiative related) in solving statistical equilibrium equations.
        
    _rel_flux : np.double, np.array with shape (_atom.nLine,)
        relative flux under the assumption of optically thin. 
        :math:`h \nu n_{j} A_{ji}`, [:math:`erg \; cm^{-3} \; s^{-1}`]
        
    Notes
    -----

    The Statistical equilibrium equations [1]_.
    
    .. math:: \bm{A} \times \bm{n} = \bm{b}
    
    .. math:: \bm{b} = (0,0,0,...,1)^{T}
    
    .. math:: A_{ii} = \sum_{j \neq i} (R_{ij} + C_{ij})
    
    .. math:: A_{ij} = -(R_{ji} + C_{ji}) \quad j \neq i, \; i \neq -1
    
    .. math:: A_{-1,j} = 1
    
    
    The absolute flux under the assumption of optically thin [2]_.
    
    .. math:: F_{ji} = \frac{1}{4 \pi R^{2}} \int_{\Delta V} \epsilon_{ji} dV \quad [erg \; cm^{-2} \; s^{-1}]
    
    where the emissivity :math:`\epsilon_{ji}` is
    
    .. math:: \epsilon_{ji} = h \nu n_{j} A_{ji} \quad [erg \; cm^{-3} \; s^{-1}]
    
    and 
    
    .. math:: \epsilon_{ji} = \int_{\nu} \epsilon_{\nu} d \nu = \int_{\nu} h \nu n_{j} A_{ji} \psi d \nu
    
    References
    ----------

    .. [1] Ivan Hubeny, Dimitri Mihalas, "Theory of Stellar Atmosphere:
        An Introduction to Astrophysical Non-equilibrium
        Quantitative Spectroscopic Analysis",
        Princeton University Press, pp. 283, 2015.
        
    .. [2] John T. Mariska, "The Solar Transition Region", 
        Cambridge University Press, pp. 19, 1992
        
    Examples
    --------
        
        >>> from src import AtomCls, LTELib, ColExcite, SEsolver, OpticallyThin
        >>> file = "/path/to/C_III_Be_like.txt"
        >>> atom = AtomCls.Atom(file)
        >>> n_LTE, n_SE, nj_SE, Cmat, Rmat, rel_flux = main_func(_atom=atom, _Te=3E+04, _Ne=1E+10)
    
    """
    #--- 
    # compute LTE population ratio for each CE transition
    #---
    
    _n_LTE = LTELib.get_LTE_ratio(_erg=_atom.Level.erg[:], _g=_atom.Level.g[:],
                                  _stage=_atom.Level.stage[:], _Te=_Te, _Ne=_Ne)
    
    #--- 
    # we are going to use LTE population ratio to convert C_{ij} to C_{ji},
    # so we need the the LTE population of the lower/upper level of each transition
    #---
    _nTran = _atom.nLine
    _ni_LTE = np.empty(_nTran, dtype=np.double)
    _nj_LTE = np.empty(_nTran, dtype=np.double)
    
    for k in range(_nTran):
        _ni_LTE[k] = _n_LTE[_atom.CE_coe.idxI[k]]
        _nj_LTE[k] = _n_LTE[_atom.CE_coe.idxJ[k]]
    
        
    #--- 
    # compute collision excitation/de-excitation rate coefficient
    #---
    
    _CE_fac = ColExcite.interpolate_CE_fac(_table=atom.CE_table[:,:], _Te=_Te, 
                                           _Te_table=_atom.CE_Te_table[:], 
                                           _f1=_atom.CE_coe.f1[:], _f2=_atom.CE_coe.f2[:])
    
    _CEij = ColExcite.get_CE_rate_coe(_CE_fac=_CE_fac, _Te=_Te, _gi=_atom.CE_coe.gi[:],
                                      _dEij=_atom.CE_coe.dEij[:], _type=_atom.CE_type)

    _CEji = ColExcite.Cij_to_Cji(_Cij=_CEij,  _ni_LTE=_ni_LTE, _nj_LTE=_nj_LTE)
    
    #--- 
    # compute collision ionization rate coefficient
    #---
    
    
    #---
    # solve SE equations under the assumption of "Corona equilibrium"
    # that is, mean intensity = 0
    #---
    
    #_nLevel = _atom.nLevel
    
    #_Cmat = np.zeros((_nLevel, _nLevel), dtype=np.double)
    #SEsolver.setMatrixC(_Cmat=_Cmat[:,:], _Cji=_CEji[:], _Cij=_CEij[:],
    #                    _idxI=_atom.CE_coe.idxI[:], _idxJ=_atom.CE_coe.idxJ[:], _Ne=_Ne)
    
    #_Rmat     = np.zeros((_nLevel, _nLevel), dtype=np.double)
    #_Rji_stim = np.zeros(_atom.Line.AJI[:].shape, dtype=np.double)
    #_Rij      = np.zeros(_atom.Line.AJI[:].shape, dtype=np.double)
    #SEsolver.setMatrixR(_Rmat=_Rmat[:,:], _Rji_spon=_atom.Line.AJI[:],
    #                    _Rji_stim=_Rji_stim[:], _Rij=_Rij, 
    #                    _idxI=_atom.Line.idxI[:], _idxJ=_atom.Line.idxJ[:])

    #_n_SE = SEsolver.solveSE(_Rmat=_Rmat[:,:], _Cmat=_Cmat[:,:])
    
    #---
    # compute relative flux for each transition
    #---
    #_nj_SE = np.empty(_nTran, dtype=np.double)
    #for k in range(_nTran):
    #    _nj_SE[k] = _n_SE[_atom.CE_coe.idxJ[k]]
    
    #_rel_flux = Thin.get_relative_flux(_AJI=_atom.Line.AJI[:], 
    #                                   _f0=_atom.Line.f0[:], _nj=_nj_SE[:])
    
    #return _n_LTE, _n_SE, _nj_SE, _Cmat, _Rmat, _rel_flux 



In [ ]:
z = 0
one_depth(_atom=atom, 
          _Te=slab1d.data.Te[z],_Ne=slab1d.data.Ne[z],
          _Vt=slab1d.data.Vt[z], _Vd=slab1d.data.Vd[z])